In [ ]:
!pip install node2vec
!pip install --upgrade gensim
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import pandas as pd
import numpy as np 
import random
random.seed(2022)
np.random.seed(2022)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data_10000_20.csv to data_10000_20.csv
Saving edge_index_data_20.csv to edge_index_data_20.csv


In [ ]:
uploaded2 = files.upload()

Saving deepwalk_10d_x20.model to deepwalk_10d_x20.model
Saving Node2Vec_10d_x20.model to Node2Vec_10d_x20.model


In [ ]:
uploaded3 = files.upload()

Saving feats_ate_x20.xlsx to feats_ate_x20.xlsx


In [ ]:
import io
df=pd.read_csv(io.BytesIO(uploaded['data_10000_20.csv']),index_col=0)
df=df.rename(columns={"20": "T", "21": "y","22":"tau"})

from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df,test_size = 0.2,random_state=42)
df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,T,y,tau,23,24
0,0.822826,0.058922,0.007085,0.437839,0.378267,0.103993,0.025474,0.044342,0.324058,0.704253,...,0.311220,0.879229,0.561044,0.128196,0.344506,1.0,1.195390,0.440874,1.264626,0.151724
1,0.541298,0.670322,0.145562,0.450309,0.819929,0.903168,0.328828,0.712272,0.542630,0.857734,...,0.890697,0.372880,0.476096,0.103108,0.171376,1.0,2.116378,0.605810,2.020121,0.900000
2,0.679938,0.462617,0.187484,0.206915,0.600526,0.864546,0.339516,0.946365,0.402442,0.530432,...,0.255293,0.278260,0.104923,0.874021,0.675286,0.0,1.255315,0.571277,1.537541,0.835032
3,0.777054,0.244043,0.358487,0.082308,0.197206,0.836733,0.743349,0.908650,0.134280,0.523403,...,0.898311,0.153092,0.642399,0.315128,0.809866,1.0,1.551284,0.510548,0.782097,0.561134
4,0.918237,0.125650,0.123677,0.902472,0.958726,0.443750,0.279191,0.143440,0.098040,0.771201,...,0.718766,0.876797,0.011292,0.090110,0.420912,0.0,2.219453,0.521944,2.019655,0.354582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.391134,0.094091,0.016039,0.098399,0.572475,0.890806,0.712849,0.909261,0.017119,0.393975,...,0.889067,0.281810,0.531565,0.676391,0.331466,0.0,1.930412,0.242613,0.968434,0.115361
9996,0.497493,0.809645,0.077595,0.493764,0.487856,0.256397,0.232405,0.562726,0.943313,0.779282,...,0.209162,0.351497,0.841826,0.497301,0.519539,1.0,2.034655,0.653569,2.048275,0.900000
9997,0.272765,0.792536,0.260427,0.846052,0.368039,0.683828,0.755955,0.786748,0.407661,0.240224,...,0.101099,0.071966,0.447678,0.481778,0.853531,1.0,3.005015,0.532650,1.772984,0.628122
9998,0.171905,0.990760,0.212039,0.499126,0.792677,0.074631,0.688703,0.035455,0.225603,0.686410,...,0.450641,0.639408,0.473543,0.411949,0.614799,0.0,2.690296,0.581333,1.571204,0.509897


In [ ]:
col=df.columns
y_train=torch.from_numpy(np.array((df_train['y']))).reshape(df_train.shape[0],1).to(torch.float32)
y_test=torch.from_numpy(np.array(df_test['y'])).reshape(df_test.shape[0],1).to(torch.float32)
x_train=torch.from_numpy(np.array(df_train[col[:21]])).to(torch.float32)
x_test=torch.from_numpy(np.array(df_test[col[:21]])).to(torch.float32)

train_reverse=df_train[col[:21]].copy()
train_reverse["T"]=1-train_reverse["T"]
train_reverse=torch.from_numpy(np.array(train_reverse)).to(torch.float32)
test_reverse=df_test[col[:21]].copy()
test_reverse["T"]=1-test_reverse["T"]
test_reverse=torch.from_numpy(np.array(test_reverse)).to(torch.float32)

In [ ]:
from scipy.stats import entropy
def R2(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    R2 = 1 - np.sum(np.square(y_actual-y_predicted)) / np.sum(np.square(y_actual-np.mean(y_actual)))
    return R2

def MAE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MAE = np.mean(abs(y_actual-y_predicted))
    return MAE

def RMSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    RMSE = np.sqrt(np.mean(np.square(y_actual-y_predicted)))
    return RMSE

def CVRMSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    RMSE = np.sqrt(np.mean(np.square(y_actual-y_predicted)))
    CVRMSE = RMSE/np.mean(y_actual)
    return CVRMSE

def MAPE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MAPE=np.mean(abs((y_actual-y_predicted)/y_actual))
    return MAPE

def MSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MSE = np.mean(abs(y_actual-y_predicted)**2)
    return MSE

def kl_divergence(p, q):

    p = np.asarray(p, dtype=float)
    q = np.asarray(q, dtype=float)
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

def kl_divergence(y_predicted, y_actual):

    stacked_values = np.hstack((y_predicted, y_actual))
    stacked_low = np.percentile(stacked_values, 0.1)
    stacked_high = np.percentile(stacked_values, 99.9)
    bins = np.linspace(stacked_low, stacked_high, 100)

    distr = np.histogram(y_predicted, bins=bins)[0]
    distr = np.clip(distr / distr.sum(), 0.001, 0.999)
    true_distr = np.histogram( y_actual, bins=bins)[0]
    true_distr = np.clip(true_distr / true_distr.sum(), 0.001, 0.999)

    kl = entropy(distr, true_distr)
    return kl 

In [ ]:
edge_index=pd.read_csv(io.BytesIO(uploaded['edge_index_data_20.csv']))
edge_index=torch.from_numpy(np.transpose(np.array(edge_index)))

In [ ]:
ate_list=[]
ate=pd.read_excel(io.BytesIO(uploaded3["feats_ate_x20.xlsx"]),index_col=0)
for i in df.columns[:-4]:
    ate_list.append(float(ate[ate['Feature']==int(i)]["ATE"].values))

In [ ]:
from gensim.models import word2vec
model_dw=word2vec.Word2Vec.load("deepwalk_10d_x20.model")
model_n2v=word2vec.Word2Vec.load("Node2Vec_10d_x20.model")

lst_dw=[]
lst_n2v=[]
for i in range(21):
    lst_dw.append(model_dw.wv[i])
    lst_n2v.append(model_n2v.wv[i])
len(lst_n2v)

21

In [ ]:
import torch
from torch_geometric.data import InMemoryDataset, download_url

# create Mytrain Data
class Mytrain(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(x_train.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =x_train[i]
            t=torch.zeros(21,32) # 32 dimensions = 1+10+21 = node number, node embedding, weight*feature

            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(causal_weighted)
            
            for j in range(21): # for j-th feature
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec
                                                     list(causal_weighted) # weight*feature (21d), causal-weighted feature        
                                                    ))) 
              
            Y = y_train[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# create Mytest Data
class Mytest(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        
   
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(x_test.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =x_test[i]
            t=torch.zeros(21,32) # 32 dimensions = 1+10+21 = node number, node embedding, weight*feature

            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(causal_weighted)
            
            for j in range(21): # for j-th feature
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec
                                                     list(causal_weighted) # weight*feature (21d), causal-weighted feature        
                                                    ))) 

            Y = y_test[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# create Mytrain_reverse Data
class Mytrain_reverse(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]
    
    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(train_reverse.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =train_reverse[i]
            t=torch.zeros(21,32) # 32 dimensions = 1+10+21 = node number, node embedding, weight*feature
            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(causal_weighted)
            
            for j in range(21): # for j-th feature
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec
                                                     list(causal_weighted) # weight*feature (21d), causal-weighted feature        
                                                    ))) 


            Y = y_train[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])


# create Mytest_reverse Data
class Mytest_reverse(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(test_reverse.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =test_reverse[i]
            t=torch.zeros(21,32) # 32 dimensions = 1+10+21 = node number, node embedding, weight*feature
            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(causal_weighted)
            
            for j in range(21): # for j-th feature
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec
                                                     list(causal_weighted) # weight*feature (6d), causal-weighted feature        
                                                    ))) 


            Y = y_test[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)

            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
Mydata_train=Mytrain(".\mydata\MYdata_train")
Mydata_test=Mytest(".\mydata\MYdata_test")
Mytrain_reverse=Mytrain_reverse(".\mydata\MYtrain_reverse")
Mytest_reverse=Mytest_reverse(".\mydata\MYtest_reverse")

Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!


# GCN

In [ ]:
import torch
from torch_scatter import scatter_add
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(32, 64)
        self.conv2 = GCNConv(64, 10)
        self.f1 = torch.nn.Linear(210,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = x.reshape(-1,210)
        x = self.f1(x)
        x = self.f2(x)
        return x

In [ ]:
def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in train_loader:
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()
    
    
    loss=loss_all/len(Mydata_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in test_loader:
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()

    
    loss = loss_all / len(Mydata_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl

    

te_train = {}
te_valid = {}
for i in range(5):
    train_mse=[]
    train_kl=[]
    train_error=[]

    valid_mse=[]
    valid_kl=[]
    valid_error=[]

    num_epochs = 2560
    batch_size = 512
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
    crit = F.mse_loss
    train_loader = DataLoader(Mydata_train, batch_size=batch_size)
    test_loader = DataLoader(Mydata_test, batch_size=512)
    reverse_train = DataLoader(Mytrain_reverse, batch_size=512)
    reverse_test = DataLoader(Mytest_reverse, batch_size=512)
    for epoch in range(num_epochs):
        loss,r2,mse,kl=train()
        if epoch %64==0:      
            print('train_loss:')
            print(loss)

            loss,r2,mse,kl=val()
            print('test_loss:')
            print(loss)
  
    y_predicted = []
    for data in train_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_train["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_train:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_train["y_reversed"]=y_reversed
    t0=df_train[df_train["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_train[df_train["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    train_mse.append(mse)
    train_kl.append(kl)
    train_error.append(ate_error)

    print("Model %d has MSE on trainset:%f" % (i,np.mean(train_mse)),end='  ')
    print("Model %d has KL on trainset:%f" % (i,np.mean(train_kl)),end='  ')
    print("Model %d has ATE error on trainset:%f" % (i,np.mean(train_error)),end='  ')

    te_train["GCN ATE % Error",i]=np.mean(train_error)
    te_train["GCN MSE",i]=np.mean(train_mse)
    te_train["GCN KL Divergence",i]=np.mean(train_kl)

    y_predicted = []
    for data in test_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_test["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_test:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_test["y_reversed"]=y_reversed
    t0=df_test[df_test["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_test[df_test["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    valid_mse.append(mse)
    valid_kl.append(kl)
    valid_error.append(ate_error)

    print("Model %d has MSE on validset:%f" % (i,np.mean(valid_mse)),end='  ')
    print("Model %d has KL on validset:%f" % (i,np.mean(valid_kl)),end='  ')
    print("Model %d has ATE error on validset:%f" % (i,np.mean(valid_error)),end='  ')

    te_valid["GCN ATE % Error",i]=np.mean(valid_error)
    te_valid["GCN MSE",i]=np.mean(valid_mse)
    te_valid["GCN KL Divergence",i]=np.mean(valid_kl)
    print(te_valid)

R2:-0.441915  MSE:1.972229  KL:0.256592  MAE:1.122195  RMSE:1.404361  CVRMSE:0.930330  train_loss:
1.9722285289764405
R2:0.161833  MSE:1.207211  KL:0.919123  MAE:0.883918  RMSE:1.098732  CVRMSE:0.709757  test_loss:
0.002415754437446594
R2:0.116128  MSE:1.208947  KL:0.601180  MAE:0.877486  RMSE:1.099521  CVRMSE:0.728387  R2:0.193272  MSE:1.103430  KL:0.351858  MAE:0.838301  RMSE:1.050443  CVRMSE:0.695874  R2:0.216435  MSE:1.071748  KL:0.270829  MAE:0.829368  RMSE:1.035253  CVRMSE:0.685811  R2:0.221077  MSE:1.065399  KL:0.248465  MAE:0.826221  RMSE:1.032182  CVRMSE:0.683777  R2:0.233127  MSE:1.048917  KL:0.254584  MAE:0.820251  RMSE:1.024167  CVRMSE:0.678467  R2:0.240746  MSE:1.038496  KL:0.234275  MAE:0.815905  RMSE:1.019066  CVRMSE:0.675089  R2:0.242095  MSE:1.036650  KL:0.224995  MAE:0.814328  RMSE:1.018160  CVRMSE:0.674488  R2:0.243470  MSE:1.034770  KL:0.238138  MAE:0.813514  RMSE:1.017236  CVRMSE:0.673876  R2:0.244160  MSE:1.033826  KL:0.233317  MAE:0.813013  RMSE:1.016772  CVRMSE:

KeyboardInterrupt: ignored

In [ ]:
te_train,te_valid

({('GCN ATE % Error', 0): 0.0064578186860938525,
  ('GCN ATE % Error', 1): 0.036165382190898265,
  ('GCN ATE % Error', 2): 0.02313157233838181,
  ('GCN ATE % Error', 3): 0.02169293390696736,
  ('GCN KL Divergence', 0): 0.6723180241479694,
  ('GCN KL Divergence', 1): 0.6608985125106143,
  ('GCN KL Divergence', 2): 0.6311085336961669,
  ('GCN KL Divergence', 3): 0.7521139172537101,
  ('GCN MSE', 0): 0.30655562038507234,
  ('GCN MSE', 1): 0.28003871939454306,
  ('GCN MSE', 2): 0.2921350629643457,
  ('GCN MSE', 3): 0.3609203436385989},
 {('GCN ATE % Error', 0): 0.042203191470106205,
  ('GCN ATE % Error', 1): 0.027076428866166014,
  ('GCN ATE % Error', 2): 0.05327539402177872,
  ('GCN ATE % Error', 3): 0.01986852567889419,
  ('GCN KL Divergence', 0): 0.7134824617766334,
  ('GCN KL Divergence', 1): 0.6815082607572295,
  ('GCN KL Divergence', 2): 0.6314093860544899,
  ('GCN KL Divergence', 3): 0.7375606129293617,
  ('GCN MSE', 0): 0.324869930187274,
  ('GCN MSE', 1): 0.2976769862399665,
  ('G

# GAT

In [ ]:
from uuid import RFC_4122
import torch
import math
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GATConv

from torch_geometric.utils import add_self_loops,degree
from torch_geometric.datasets import Planetoid
import ssl
import torch.nn.functional as F


class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.gat1=GATConv(in_channels=32,out_channels=8,heads=8,dropout=0.6)
        self.gat2=GATConv(in_channels=64,out_channels=10,heads=1,dropout=0.6)
        self.f1 = torch.nn.Linear(210,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self,data):
        x,edge_index=data.x, data.edge_index
        x=self.gat1(x,edge_index)
        x=self.gat2(x,edge_index)
        x=x.reshape(-1,210)
        x = self.f1(x)
        x = self.f2(x)
        return x

ssl._create_default_https_context = ssl._create_unverified_context
def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in train_loader:
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()
    
    
    loss=loss_all/len(Mydata_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in test_loader:
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()

    
    loss = loss_all / len(Mydata_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl

    
for i in range(3):
    train_mse=[]
    train_kl=[]
    train_error=[]

    valid_mse=[]
    valid_kl=[]
    valid_error=[]

    num_epochs = 2560
    batch_size = 512
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
    crit = F.mse_loss
    train_loader = DataLoader(Mydata_train, batch_size=batch_size)
    test_loader = DataLoader(Mydata_test, batch_size=512)
    reverse_train = DataLoader(Mytrain_reverse, batch_size=512)
    reverse_test = DataLoader(Mytest_reverse, batch_size=512)
    for epoch in range(num_epochs):
        loss,r2,mse,kl=train()
        if epoch %64==0:      
            print('train_loss:')
            print(loss)

            loss,r2,mse,kl=val()
            print('test_loss:')
            print(loss)
  
    y_predicted = []
    for data in train_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_train["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_train:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_train["y_reversed"]=y_reversed
    t0=df_train[df_train["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_train[df_train["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    train_mse.append(mse)
    train_kl.append(kl)
    train_error.append(ate_error)

    print("Model %d has MSE on trainset:%f" % (i,np.mean(train_mse)),end='  ')
    print("Model %d has KL on trainset:%f" % (i,np.mean(train_kl)),end='  ')
    print("Model %d has ATE error on trainset:%f" % (i,np.mean(train_error)),end='  ')

    te_train["GAT ATE % Error",i]=np.mean(train_error)
    te_train["GAT MSE",i]=np.mean(train_mse)
    te_train["GAT KL Divergence",i]=np.mean(train_kl)

    y_predicted = []
    for data in test_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_test["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_test:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_test["y_reversed"]=y_reversed
    t0=df_test[df_test["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_test[df_test["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    valid_mse.append(mse)
    valid_kl.append(kl)
    valid_error.append(ate_error)

    print("Model %d has MSE on validset:%f" % (i,np.mean(valid_mse)),end='  ')
    print("Model %d has KL on validset:%f" % (i,np.mean(valid_kl)),end='  ')
    print("Model %d has ATE error on validset:%f" % (i,np.mean(valid_error)),end='  ')

    te_valid["GAT ATE % Error",i]=np.mean(valid_error)
    te_valid["GAT MSE",i]=np.mean(valid_mse)
    te_valid["GAT KL Divergence",i]=np.mean(valid_kl)


R2:-0.285108  MSE:1.757751  KL:0.143266  MAE:1.050041  RMSE:1.325802  CVRMSE:0.878288  train_loss:
1.757751085281372
R2:0.193240  MSE:1.161975  KL:0.617195  MAE:0.866952  RMSE:1.077950  CVRMSE:0.696332  test_loss:
0.0023251556754112245
R2:0.129711  MSE:1.190368  KL:0.245668  MAE:0.870200  RMSE:1.091040  CVRMSE:0.722768  R2:0.181081  MSE:1.120105  KL:0.188285  MAE:0.844909  RMSE:1.058350  CVRMSE:0.701113  R2:0.195787  MSE:1.099989  KL:0.196477  MAE:0.838495  RMSE:1.048804  CVRMSE:0.694788  R2:0.207855  MSE:1.083484  KL:0.217365  MAE:0.831531  RMSE:1.040905  CVRMSE:0.689556  R2:0.208259  MSE:1.082930  KL:0.212160  MAE:0.831130  RMSE:1.040639  CVRMSE:0.689380  R2:0.213436  MSE:1.075849  KL:0.212343  MAE:0.829542  RMSE:1.037232  CVRMSE:0.687122  R2:0.224386  MSE:1.060873  KL:0.210854  MAE:0.824825  RMSE:1.029987  CVRMSE:0.682323  R2:0.221860  MSE:1.064327  KL:0.211979  MAE:0.823699  RMSE:1.031662  CVRMSE:0.683433  R2:0.219517  MSE:1.067532  KL:0.215895  MAE:0.825015  RMSE:1.033215  CVRMSE:

In [ ]:
te_train,te_valid

({('GAT ATE % Error', 0): 0.3700368934943121,
  ('GAT ATE % Error', 1): 0.36539716628577823,
  ('GAT ATE % Error', 2): 0.3524490360193424,
  ('GAT KL Divergence', 0): 0.7041002320872455,
  ('GAT KL Divergence', 1): 0.697124255112526,
  ('GAT KL Divergence', 2): 0.6995187431119597,
  ('GAT MSE', 0): 0.08471279953542854,
  ('GAT MSE', 1): 0.08317695660820222,
  ('GAT MSE', 2): 0.07976093489422811},
 {('GAT ATE % Error', 0): 0.5730956577315547,
  ('GAT ATE % Error', 1): 0.5647502709225933,
  ('GAT ATE % Error', 2): 0.5520719678835655,
  ('GAT KL Divergence', 0): 1.2133870378550549,
  ('GAT KL Divergence', 1): 1.2173954634230697,
  ('GAT KL Divergence', 2): 1.191252474948668,
  ('GAT MSE', 0): 0.11261318813230459,
  ('GAT MSE', 1): 0.10970702301029794,
  ('GAT MSE', 2): 0.10609582293535014})

In [ ]:
gcn_error=[te_train[i] for i in te_train if 'GCN ATE % Error' in i[0]][1]
gcn_mse=[te_train[i] for i in te_train if 'GCN MSE' in i[0]][1]
gcn_kl=[te_train[i] for i in te_train if 'GCN KL Divergence' in i[0]][1]
gat_error=[te_train[i] for i in te_train if 'GAT ATE % Error' in i[0]][2]
gat_mse=[te_train[i] for i in te_train if 'GAT MSE' in i[0]][2]
gat_kl=[te_train[i] for i in te_train if 'GAT KL Divergence' in i[0]][2]
train_summary=pd.DataFrame(columns=[["Abs % Error of ATE","MSE","KL Divergence"]])
train_summary.loc["S learner(GCN)",:]=[gcn_error,gcn_mse,gcn_kl]
train_summary.loc["S learner(GAT)",:]=[gat_error,gat_mse,gat_kl]

gcn_error=[te_valid[i] for i in te_valid if 'GCN ATE % Error' in i[0]][1]
gcn_mse=[te_valid[i] for i in te_valid if 'GCN MSE' in i[0]][1]
gcn_kl=[te_valid[i] for i in te_valid if 'GCN KL Divergence' in i[0]][1]
gat_error=[te_valid[i] for i in te_valid if 'GAT ATE % Error' in i[0]][2]
gat_mse=[te_valid[i] for i in te_valid if 'GAT MSE' in i[0]][2]
gat_kl=[te_valid[i] for i in te_valid if 'GAT KL Divergence' in i[0]][2]
valid_summary=pd.DataFrame(columns=[["Abs % Error of ATE","MSE","KL Divergence"]])
valid_summary.loc["S learner(GCN)",:]=[gcn_error,gcn_mse,gcn_kl]
valid_summary.loc["S learner(GAT)",:]=[gat_error,gat_mse,gat_kl]

print("Validation Summary of Embedding(10d) & Causal Weighting(6d)")
valid_summary

Validation Summary of Embedding(10d) & Causal Weighting(6d)


,Abs % Error of ATE,MSE,KL Divergence
S learner(GCN),0.042666,0.155125,0.433525
S learner(GAT),0.440617,0.090175,0.813079


In [ ]:
train_summary.to_csv("Train_embedding(10d)_ate(10d)_x20.csv")
valid_summary.to_csv("Valid_embedding(10d)_ate(10d)_x20.csv")